In [5]:
!pip install ujson
!pip install json
!pip install sentence-transformers
!pip install dspy
from sentence_transformers import SentenceTransformer
import ujson
import dspy
from dspy.utils import download


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [4]:
download("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json")
%pip install -U faiss-cpu  # or faiss-gpu if you have a GPU

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer

#从HF镜像下载到本地，https://hf-mirror.com/
#huggingface-cli download moka-ai/m3e-base --local-dir ./m3e-base --local-dir-use-symlinks False
model = SentenceTransformer('./m3e-base', device='cuda')

In [6]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

# huggingface-cli download dspy/cache ragqa_arena_tech_corpus.jsonl --revision main --local-dir .
with open("./ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

# Encode the corpus



embedder = dspy.Embedder(model.encode)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)


Loaded 28436 documents. Will encode them below.
Training a 32-byte FAISS index with 337 partitions, based on 28436 x 768-dim embeddings


In [10]:
import dspy
import os
from dspy import LM
# get deepseek's API key from the .env file
from dotenv import load_dotenv
load_dotenv()
# get the API key
api_key = os.getenv("DEEPSEEK_API_KEY")
# set the API key

# set the LM to use

lm = dspy.LM("deepseek/deepseek-chat")
dspy.configure(lm=lm, api_key=api_key)


In [12]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)
    
rag = RAG()
rag(question="what is the difference between mac and windows?").response
dspy.inspect_history()





[2025-04-17T19:21:48.252660]

System message:

Your input fields are:
1. `context` (str)
2. `question` (str)
Your output fields are:
1. `reasoning` (str)
2. `response` (str)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
[1] «The OpenGL implementation for MacOS is MacOS — OpenGL support is built in. https://developer.apple.com/opengl/»
[2] «Theres no exact equivalent thats distributed with Mac OS X, but there are a couple other ways to get the information: system_profiler - Shows all of the Macs System Profile info in the shell sysctl -a | grep mem or sysctl hw.memsize (total mem)»
[3] «I recommend creating a ExFat p